In [1]:
%tensorflow_version 1.x
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/content/",one_hot=True)
# Please use alternatives such as official/mnist/dataset.py from tensorflow/models.

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /content/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /content/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /content/t10k-images-idx3-ubyte.gz
Extracting /content/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
import tensorflow as tf

# ANN

In [3]:
n_classes = 10
n_hidden_neurons = 500

X = tf.placeholder(tf.float32,shape=[None,784])
y = tf.placeholder(tf.float32,shape=[None,10])

    
hidden_layer1 = {
    "weights" :tf.Variable(tf.random_normal([784,n_hidden_neurons])),
    "bias" : tf.Variable(tf.random_normal([n_hidden_neurons]))
}

output_layer = {
    "weights" :tf.Variable(tf.random_normal([n_hidden_neurons,n_classes])),
    "bias" : tf.Variable(tf.random_normal([n_classes]))
}


def neural_network(data):
    
  l1 = tf.matmul(data,hidden_layer1['weights'])+hidden_layer1['bias']
  l1 = tf.nn.relu(l1)

  output = tf.matmul(l1,output_layer['weights'])+output_layer['bias']

  return output

def training_nn(X):
    
  y_pred = neural_network(X)
  error = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=y_pred))

  optimiser = tf.train.AdamOptimizer(learning_rate=0.01).minimize(error)

  init  = tf.global_variables_initializer()

  saver = tf.train.Saver()
  with tf.Session() as sess:
    sess.run(init)

    for steps in range(1000):    
        
      batch_x, batch_y = mnist.train.next_batch(300)

      sess.run(optimiser,feed_dict = {X:batch_x,y:batch_y})

    # saver.save(sess,"/content/Mnist_10K_adam.h5")

    # with tf.Session() as sess:
      
    # saver.restore(sess,"/content/Mnist_10K_adam.h5")
    correct_pred  = tf.equal(tf.argmax(y_pred,1),tf.argmax(y,1))

    accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

    print("Accuracy : ",sess.run(accuracy,feed_dict={X:mnist.test.images,y:mnist.test.labels}))

training_nn(X)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Accuracy :  0.9564


# CNN


In [4]:
n_classes = 10
n_hidden_neurons = 500

drop_rate = 0.8

X = tf.placeholder(tf.float32,shape=[None,784])
y = tf.placeholder(tf.float32,shape=[None,10])

def cnn_layer(data):

  weights = {
      "W_convo1" : tf.Variable(tf.random_normal([5,5,1,32])),
      "W_convo2" : tf.Variable(tf.random_normal([5,5,32,64])),
      "W_fc" : tf.Variable(tf.random_normal([7*7*64,1024])),
      "W_output" : tf.Variable(tf.random_normal([1024,n_classes])),


  }

  biases = {
      "B_convo1" : tf.Variable(tf.random_normal([32])),
      "B_convo2" : tf.Variable(tf.random_normal([64])),
      "B_fc" : tf.Variable(tf.random_normal([1024])),
      "B_output" : tf.Variable(tf.random_normal([n_classes])),

  }

  data = tf.reshape(data,shape=[-1,28,28,1])

  convo2d_layer1 = tf.nn.relu(tf.nn.conv2d(data,weights['W_convo1'],strides=[1,1,1,1],padding="SAME")
                  + biases["B_convo1"])
  max_pooling1  = tf.nn.max_pool(convo2d_layer1,ksize=[1,2,2,1] , strides=[1,2,2,1],padding="SAME")

  convo2d_layer2 = tf.nn.relu(tf.nn.conv2d(max_pooling1,weights['W_convo2'],strides=[1,1,1,1],padding="SAME")
                  + biases["B_convo2"])
  max_pooling2  = tf.nn.max_pool(convo2d_layer2,ksize=[1,2,2,1], strides=[1,2,2,1],padding="SAME")

  fc = tf.reshape(max_pooling2,[-1,7*7*64])

  fc_nn = tf.nn.relu(tf.matmul(fc,weights['W_fc'])+ biases['B_fc'])

  # dropout = tf.nn.dropout(fc_nn,keep_prob=keep_rate)

  output = tf.matmul(fc_nn,weights['W_output']) + biases['B_output']

  return output

def training_nn(X):
    
  y_pred = cnn_layer(X)
  error = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=y_pred))

  optimiser = tf.train.AdamOptimizer().minimize(error)

  init  = tf.global_variables_initializer()

  saver = tf.train.Saver()


  with tf.Session() as sess:
    sess.run(init)

    for steps in range(1000):    
        
      batch_x, batch_y = mnist.train.next_batch(300)

      sess.run(optimiser,feed_dict = {X:batch_x,y:batch_y})
      
    correct_pred  = tf.equal(tf.argmax(y_pred,1),tf.argmax(y,1))

    accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

    print("Accuracy : ",sess.run(accuracy,feed_dict={X:mnist.test.images,y:mnist.test.labels}))


training_nn(X)

Accuracy :  0.9632


# RNN

In [5]:
from tensorflow.python.ops import rnn, rnn_cell

tf.reset_default_graph()

n_classes = 10

n_chunks = 28
chunk_size = 28
rnn_size =128

X = tf.placeholder(tf.float32,shape=[None,chunk_size,n_chunks])
y = tf.placeholder(tf.float32,shape=[None,n_classes])

def rnn_layer(X):

  rnn_layer = {
      
      "weights" : tf.Variable(tf.random_normal([rnn_size,n_classes])),
      "bias" : tf.Variable(tf.random_normal([n_classes]))
  }

  X = tf.transpose(X,perm=[1,0,2])
  X = tf.reshape(X,[-1,chunk_size])
  X = tf.split(X,n_chunks,0)


  lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size,state_is_tuple=True)
  outputs, states = rnn.static_rnn(lstm_cell,X,dtype=tf.float32)

  output_layer = tf.matmul(outputs[-1],rnn_layer['weights'])+ rnn_layer['bias']
  return output_layer

def training_nn(X):
    
  y_pred = rnn_layer(X)
  error = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=y_pred))

  optimiser = tf.train.AdamOptimizer().minimize(error)

  init  = tf.global_variables_initializer()

  saver = tf.train.Saver()


  with tf.Session() as sess:
    sess.run(init)

    for steps in range(1500):    
        
      batch_x, batch_y = mnist.train.next_batch(128)

      batch_x = batch_x.reshape((128,n_chunks,chunk_size))

      sess.run(optimiser,feed_dict = {X:batch_x,y:batch_y})
      
    correct_pred  = tf.equal(tf.argmax(y_pred,1),tf.argmax(y,1))

    accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

    print("Accuracy : ",sess.run(accuracy,feed_dict={X:mnist.test.images.reshape((-1, n_chunks, chunk_size)),y:mnist.test.labels}))



training_nn(X)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Accuracy :  0.9772
